# Battle of cities
*By David Zakarias (david.zakarias at gmail.com) &copy; 2020*

My submission for the IBM Data Science Capstone course on Coursera.
A data science project to find the best candidate city for a new software R&D office.

In [65]:
import numpy as np
import pandas as pd
import requests
import folium
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import json

# Climate

## Gathering climate data from Wikipedia
We will fetch climate data from Wikipedia for US and European cities. We'll put them into clusters, and visualize the resulting clusters. It will help getting an idea what the seasons are like in U.S. cities for a European audience.

In [3]:
wikiurl = 'https://en.wikipedia.org/wiki/'

def getAvgTempData(continent, country=None):
    df = pd.read_html(f'{wikiurl}List_of_cities_by_average_temperature', match = continent)[0]
    if country:
        df = df[df['Country'] == country]
    df.drop(['Country', 'Ref.'], axis=1, inplace=True)
    df.set_index('City', inplace=True)
    return df

us_avg_temp = getAvgTempData('North America', 'United States')
print(us_avg_temp.index)
us_avg_temp.head()

Index(['Albuquerque', 'Anchorage', 'Atlanta', 'Austin', 'Baltimore', 'Boise',
       'Boston', 'Charlotte', 'Chicago', 'Columbus', 'Dallas', 'Denver',
       'Detroit', 'El Paso', 'Fairbanks', 'Houston', 'Indianapolis',
       'Jacksonville', 'Kansas City', 'Las Vegas', 'Lake Havasu City',
       'Los Angeles', 'Louisville', 'Memphis', 'Miami', 'Milwaukee',
       'Minneapolis', 'Nashville', 'New Orleans', 'New York City',
       'OklahomaCity', 'Omaha', 'Philadelphia', 'Phoenix', 'Pittsburgh',
       'Portland (OR)', 'Sacramento', 'Salt LakeCity', 'San Antonio',
       'San Diego', 'SanFrancisco', 'Seattle', 'St. Louis', 'Tampa', 'Tucson',
       'VirginiaBeach', 'Washington,D.C.', 'Wichita'],
      dtype='object', name='City')


,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
City,,,,,,,,,,,,,
Albuquerque,2.4(36.4),5.2(41.4),8.9(48.1),13.3(56.0),18.7(65.6),23.8(74.9),25.7(78.3),24.6(76.2),20.7(69.3),14.2(57.5),7.2(44.9),2.4(36.3),14.0(57.2)
Anchorage,−8.3(17.1),−6.6(20.2),−3.0(26.6),2.7(36.8),8.8(47.8),12.9(55.2),14.9(58.8),13.7(56.7),9.2(48.6),1.6(34.8),−5.4(22.2),−7.2(19.0),2.8(37.1)
Atlanta,6.3(43.3),8.4(47.1),12.3(54.2),16.4(61.5),21.1(69.9),25.2(77.4),26.9(80.5),26.3(79.4),22.9(73.2),17.1(62.8),12.0(53.6),7.3(45.1),16.8(62.3)
Austin,10.8(51.5),12.8(55.0),16.5(61.7),20.7(69.2),24.8(76.6),27.9(82.2),29.4(85.0),29.9(85.8),26.7(80.0),21.8(71.2),16.1(61.0),11.4(52.5),20.7(69.3)
Baltimore,0.8(33.5),2.4(36.4),6.8(44.2),12.4(54.3),17.6(63.6),22.8(73.0),25.3(77.6),24.3(75.7),20.2(68.4),13.7(56.7),8.3(47.0),2.9(37.3),13.1(55.6)


Clearly we'll need some additional data wrangling: the data values are strings, and what's worse, they include temperatures in both Celsius and Fahrenheit (in parentheses). As the target audience is European, only Celsius should remain and we should convert the strings into floats. Also some names are missing spaces (e.g. "OklahomaCity"), we need to fix those as well for geolocation to work later on.

In [4]:
def delete_imperial_measures(df):
    return df.replace(to_replace='([0-9,\.]+)\((.*)\)', value=r'\1', regex=True)

def wiki_df_to_float(df):
    return df.replace({'−': '-'}, regex=True).astype(float)

us_avg_temp = delete_imperial_measures(us_avg_temp)
us_avg_temp = wiki_df_to_float(us_avg_temp)

us_avg_temp.rename(index={'OklahomaCity': 'Oklahoma City', 'Portland (OR)': 'Portland, OR',
                          'Salt LakeCity': 'Salt Lake City', 'SanFrancisco': 'San Francisco',
                          'VirginiaBeach': 'Virginia Beach'}, inplace=True)

print(us_avg_temp.index)
us_avg_temp.head()



Index(['Albuquerque', 'Anchorage', 'Atlanta', 'Austin', 'Baltimore', 'Boise',
       'Boston', 'Charlotte', 'Chicago', 'Columbus', 'Dallas', 'Denver',
       'Detroit', 'El Paso', 'Fairbanks', 'Houston', 'Indianapolis',
       'Jacksonville', 'Kansas City', 'Las Vegas', 'Lake Havasu City',
       'Los Angeles', 'Louisville', 'Memphis', 'Miami', 'Milwaukee',
       'Minneapolis', 'Nashville', 'New Orleans', 'New York City',
       'Oklahoma City', 'Omaha', 'Philadelphia', 'Phoenix', 'Pittsburgh',
       'Portland, OR', 'Sacramento', 'Salt Lake City', 'San Antonio',
       'San Diego', 'San Francisco', 'Seattle', 'St. Louis', 'Tampa', 'Tucson',
       'Virginia Beach', 'Washington,D.C.', 'Wichita'],
      dtype='object', name='City')


,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
City,,,,,,,,,,,,,
Albuquerque,2.4,5.2,8.9,13.3,18.7,23.8,25.7,24.6,20.7,14.2,7.2,2.4,14.0
Anchorage,-8.3,-6.6,-3.0,2.7,8.8,12.9,14.9,13.7,9.2,1.6,-5.4,-7.2,2.8
Atlanta,6.3,8.4,12.3,16.4,21.1,25.2,26.9,26.3,22.9,17.1,12.0,7.3,16.8
Austin,10.8,12.8,16.5,20.7,24.8,27.9,29.4,29.9,26.7,21.8,16.1,11.4,20.7
Baltimore,0.8,2.4,6.8,12.4,17.6,22.8,25.3,24.3,20.2,13.7,8.3,2.9,13.1


Much better. A quick sanity check to see we got what we wanted:

In [5]:
us_avg_temp['Jan'].mean()

3.7208333333333337

In [6]:
eu_avg_temp = getAvgTempData('Europe')
eu_avg_temp = delete_imperial_measures(eu_avg_temp)
eu_avg_temp = wiki_df_to_float(eu_avg_temp)

In [7]:
eu_avg_temp.head()

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
City,,,,,,,,,,,,,
Tirana,6.7,7.8,10.0,13.4,18.0,21.6,24.0,23.8,20.7,16.0,11.7,8.1,15.2
Vienna,0.3,1.5,5.7,10.7,15.7,18.7,20.8,20.2,15.4,10.2,5.1,1.1,10.4
Minsk,-4.5,-4.4,0.0,7.2,13.3,16.4,18.5,17.5,12.1,6.6,0.6,-3.4,6.7
Brussels,3.3,3.7,6.8,9.8,13.6,16.2,18.4,18.0,14.9,11.1,6.8,3.9,10.5
Sarajevo,-0.5,1.4,5.7,10.0,14.8,17.7,19.7,19.7,15.3,11.0,5.4,0.9,10.1


In [8]:
eu_avg_temp.index

Index(['Tirana', 'Vienna', 'Minsk', 'Brussels', 'Sarajevo', 'Sofia', 'Zagreb',
       'Nicosia', 'Prague', 'Copenhagen', 'Tallinn', 'Helsinki', 'Kuopio',
       'Oulu', 'Marseille', 'Paris', 'Berlin', 'Frankfurt', 'Athens',
       'Piraeus', 'Budapest', 'Reykjavík', 'Dublin', 'Milan', 'Rome', 'Riga',
       'Vaduz', 'Vilnius', 'Luxembourg City', 'Skopje', 'Valletta', 'Chișinău',
       'Monaco', 'Podgorica', 'Amsterdam', 'Bergen', 'Oslo', 'Tromsø',
       'Warsaw', 'Lisbon', 'Bucharest', 'Arkhangelsk', 'Moscow', 'Murmansk',
       'Rostov-on-Don', 'Saint Petersburg', 'Sochi', 'Belgrade', 'Bratislava',
       'Ljubljana', 'Barcelona', 'Las Palmas de Gran Canaria', 'Madrid',
       'Seville', 'Stockholm', 'Zürich', 'Istanbul', 'Kiev', 'Lviv', 'Odessa',
       'Edinburgh', 'London'],
      dtype='object', name='City')

A quick discussion with stakeholders reveals that they especially like some other European cities' climate. Examples are Malaga, Luzern, Nice and Dubrovnik. Unfortunately the dataframe we fetched do not have those cities. However, the individual pages of the cities do have that information. Therefore we put together a new function for fetching climate data from those pages.

In [9]:
def getClimateData(city):    
    cityurl = city.replace(' ', '_')
    df = pd.read_html(f'{wikiurl}{cityurl}', match ='Climate data', header=1, index_col='Month')[0]
    df.index.rename('Stat', inplace=True)
    df = delete_imperial_measures(df)
    #delete rows that do not contain numbers
    df = df.apply(lambda x: pd.to_numeric(x, errors='coerce')).dropna()
    df = df.rename(index=lambda x: x.replace(' (°F)',''))
    return df

nice = getClimateData('Nice')

In [10]:
nice

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
Stat,,,,,,,,,,,,,
Record high °C,22.5,25.8,26.1,26.0,30.3,36.8,37.0,37.7,33.9,29.9,25.4,22.0,37.7
Average high °C,13.1,13.4,15.2,17.0,20.7,24.3,27.3,27.7,24.6,21.0,16.6,13.8,19.6
Daily mean °C,9.2,9.6,11.6,13.6,17.4,20.9,23.8,24.1,21.0,17.4,12.9,10.0,16.0
Average low °C,5.3,5.9,7.9,10.2,14.1,17.5,20.3,20.5,17.3,13.7,9.2,6.3,12.4
Average precipitation mm (inches),69.0,44.7,38.7,69.3,44.6,34.3,12.1,17.8,73.1,132.8,103.9,92.7,733.0
Average precipitation days (≥ 1.0 mm),5.8,4.7,4.6,7.1,5.2,3.8,1.8,2.4,4.9,7.2,7.2,6.4,61.2
Average snowy days,0.4,0.6,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,1.2
Mean monthly sunshine hours,157.7,171.2,217.5,224.0,267.1,306.1,347.5,315.8,242.0,187.0,149.3,139.3,2724.2
Average ultraviolet index,1.0,2.0,4.0,5.0,7.0,8.0,8.0,7.0,5.0,3.0,2.0,1.0,4.0


We only need the row 'Daily mean °C'. Append this to our dataframe:

In [11]:
eu_avg_temp = eu_avg_temp.append(nice.loc['Daily mean °C'].rename('Nice'))

Let's write a function for doing this to the remaining three cities mentioned above.

In [12]:
def append_to_df_city_climate_data(df, cities, stat_name):
    for city in cities:
        city_df = getClimateData(city)
        df = df.append(city_df.loc[stat_name].rename(city))
    return df

In [13]:
eu_avg_temp = append_to_df_city_climate_data(eu_avg_temp, ['Malaga', 'Luzern', 'Dubrovnik'], 'Daily mean °C')
eu_avg_temp

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
City,,,,,,,,,,,,,
Tirana,6.7,7.8,10.0,13.4,18.0,21.6,24.0,23.8,20.7,16.0,11.7,8.1,15.2
Vienna,0.3,1.5,5.7,10.7,15.7,18.7,20.8,20.2,15.4,10.2,5.1,1.1,10.4
Minsk,-4.5,-4.4,0.0,7.2,13.3,16.4,18.5,17.5,12.1,6.6,0.6,-3.4,6.7
Brussels,3.3,3.7,6.8,9.8,13.6,16.2,18.4,18.0,14.9,11.1,6.8,3.9,10.5
Sarajevo,-0.5,1.4,5.7,10.0,14.8,17.7,19.7,19.7,15.3,11.0,5.4,0.9,10.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
London,4.3,4.5,6.9,8.7,12.1,15.1,17.3,17.0,14.3,10.9,7.2,4.7,10.3
Nice,9.2,9.6,11.6,13.6,17.4,20.9,23.8,24.1,21.0,17.4,12.9,10.0,16.0
Malaga,12.1,12.9,14.7,16.3,19.3,23.0,25.5,26.0,23.5,19.5,15.7,13.2,18.5


Excellent! Now we have all cities we need.

## Create clusters
We will apply K-Means Clustering to form groups of cities. Seeing which European cities are in the same cluster as a candidate U.S. city, we'll get an idea about climate in that city.

In [14]:
kclusters = 6

city_avg_temp_clustering = pd.concat([eu_avg_temp.drop('Year', 1), us_avg_temp.drop('Year', 1)])
city_avg_temp_clustering

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
City,,,,,,,,,,,,
Tirana,6.7,7.8,10.0,13.4,18.0,21.6,24.0,23.8,20.7,16.0,11.7,8.1
Vienna,0.3,1.5,5.7,10.7,15.7,18.7,20.8,20.2,15.4,10.2,5.1,1.1
Minsk,-4.5,-4.4,0.0,7.2,13.3,16.4,18.5,17.5,12.1,6.6,0.6,-3.4
Brussels,3.3,3.7,6.8,9.8,13.6,16.2,18.4,18.0,14.9,11.1,6.8,3.9
Sarajevo,-0.5,1.4,5.7,10.0,14.8,17.7,19.7,19.7,15.3,11.0,5.4,0.9
...,...,...,...,...,...,...,...,...,...,...,...,...
Tampa,15.9,17.3,19.5,22.1,25.7,27.8,28.2,28.3,27.5,24.4,20.6,17.2
Tucson,11.6,13.1,15.8,19.7,24.6,29.6,30.8,29.8,27.7,21.9,15.6,11.3
Virginia Beach,4.9,6.1,9.8,14.8,19.4,24.3,26.7,25.7,22.6,16.9,11.8,6.9


In [15]:
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(city_avg_temp_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([4, 3, 5, 3, 3, 0, 3, 1, 3, 3, 5, 5, 2, 2, 4, 3, 3, 3, 1, 1, 0, 5,
       3, 0, 4, 5, 3, 5, 3, 0, 1, 0, 4, 4, 3, 3, 5, 5, 3, 4, 0, 2, 5, 2,
       0, 5, 4, 0, 0, 0, 4, 1, 4, 1, 5, 3, 4, 5, 5, 0, 3, 3, 4, 1, 3, 4,
       0, 2, 4, 1, 0, 0, 0, 4, 0, 0, 1, 0, 0, 4, 2, 1, 0, 1, 0, 1, 1, 4,
       0, 4, 1, 0, 5, 4, 1, 0, 4, 0, 0, 1, 0, 3, 4, 0, 1, 4, 4, 3, 0, 1,
       1, 4, 4, 0])

Let's put these clusters on a map!

In [ ]:
# Let's grab city coordinates
def do_geocode(geolocator, address):
    try:
        return geolocator.geocode(address)
    except GeocoderTimedOut:
        return do_geocode(geolocator, address)

coordinates = []
geolocator = Nominatim(user_agent="world_explorer")
for city in city_avg_temp_clustering.index:    
    location = do_geocode(geolocator, city)    
    coordinates.append([location.latitude, location.longitude])

In [17]:
# create map
map_clusters = folium.Map(location=[40, -40], zoom_start=3)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(np.asarray(coordinates)[:,0], np.asarray(coordinates)[:,1], city_avg_temp_clustering.index, kmeans.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Interesting results. **Cluster 0** is Central-Eastern Europe and many U.S. cities from Salt Lake City through New York until Boston. **Cluster 1** is the South-Mediterranean from Seville, Malaga on the West to Nicosia on the East, plus the Canaries. In the U.S. most southern cities (except California) also fall into this cluster which might be surprising.
**Cluster 2** is snow and ice: Oulu, Murmansk, Arhangelsk in Europe, Alaska in the U.S.
**Cluster 3**: A large part of Western Europe belongs here, the UK, Northern France, Germany. Interesting that very few cities in the U.S. have this climate: only Seattle and Portland in the North-West. **Cluster 4** is nice weather: the North Mediterranean plus Istanbul, Lisbon and Madrid in Europe, California, and the cities north to Cluster 1 and south to Cluster 0 in the US from El Paso through Memphis to Virginia Beach. The **last cluster (#5)** is Scandinavian cities Reykjavik, Oslo, Stockholm plus the cold Northeastern parts of Europe: the Baltic states, Ukraine, Belarus and Moscow. Not surprising that the U.S. doesn't have many entries here: only Minneapolis.

# Quality of life
We'll be examining the The **Movehub City Rankings** dataset, fetched from **Kaggle** (https://www.kaggle.com/blitzr/movehub-city-rankings).  This dataset has valuable features such as Health Care index, Pollution index and a more general and ambitious-sounding “Quality of Life” index for major cities around the world. We’ll just concentrate on U.S. cities.


In [20]:
qol = pd.read_csv('movehubqualityoflife.csv')
qol.set_index('City', inplace=True)
qol.head()

,Movehub Rating,Purchase Power,Health Care,Pollution,Quality of Life,Crime Rating
City,,,,,,
Caracas,65.18,11.25,44.44,83.45,8.61,85.70
Johannesburg,84.08,53.99,59.98,47.39,51.26,83.93
Fortaleza,80.17,52.28,45.46,66.32,36.68,78.65
Saint Louis,85.25,80.40,77.29,31.33,87.51,78.13
Mexico City,75.07,24.28,61.76,18.95,27.91,77.86


Let's join our previous U.S. dataset containing the monthly average temperatures with this dataset.

In [21]:
us_cities = pd.merge(qol, us_avg_temp.drop('Year', 1), left_index=True, right_index=True)
us_cities

,Movehub Rating,Purchase Power,Health Care,Pollution,Quality of Life,Crime Rating,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
City,,,,,,,,,,,,,,,,,,
Detroit,70.63,73.81,63.05,83.45,50.99,76.69,-3.7,-2.3,2.7,9.4,15.2,20.7,23.0,22.1,17.9,11.2,5.2,-1.2
Las Vegas,84.88,80.46,66.52,59.73,60.50,70.57,8.8,11.1,15.0,19.1,24.7,29.9,33.2,32.1,27.6,20.4,13.1,8.2
Philadelphia,83.31,68.77,54.17,38.64,65.53,68.58,0.5,2.0,6.4,12.2,17.7,22.9,25.6,24.7,20.6,14.2,8.7,3.0
Los Angeles,86.86,62.75,68.61,75.20,62.82,65.74,14.1,14.3,14.8,16.0,17.4,19.0,20.8,21.2,20.9,19.2,16.5,14.1
Miami,84.43,57.79,64.44,22.45,74.77,64.85,19.9,21.1,22.4,24.2,26.5,28.1,28.8,28.9,28.2,26.4,23.7,21.2
Houston,85.24,71.96,79.44,65.21,74.08,61.72,11.5,13.3,16.9,20.7,24.8,27.7,28.9,29.0,26.3,21.7,16.6,12.3
Tampa,84.27,75.72,56.94,49.89,80.75,60.73,15.9,17.3,19.5,22.1,25.7,27.8,28.2,28.3,27.5,24.4,20.6,17.2
Dallas,84.71,76.18,69.99,83.89,80.56,59.90,7.7,9.9,14.2,18.6,23.3,27.4,29.7,29.8,25.6,19.8,13.7,8.4
Baltimore,84.28,78.78,72.08,77.98,82.00,58.85,0.8,2.4,6.8,12.4,17.6,22.8,25.3,24.3,20.2,13.7,8.3,2.9


Nice. We immediately notice the Pollution value 0.00 for Nashville. It can't be right, it seems this data point is missing. Let's replace it with the mean Pollution value.

In [22]:
us_cities.loc['Nashville']['Pollution'] = us_cities.drop('Nashville')['Pollution'].mean()
us_cities.loc['Nashville']['Pollution']

52.52318181818182

Let's sort our dataset by Quality of Life:

In [23]:
us_cities.sort_values('Quality of Life', ascending=False, inplace=True)
us_cities.head()

,Movehub Rating,Purchase Power,Health Care,Pollution,Quality of Life,Crime Rating,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
City,,,,,,,,,,,,,,,,,,
Austin,84.86,69.22,73.61,28.84,86.51,42.50,10.8,12.8,16.5,20.7,24.8,27.9,29.4,29.9,26.7,21.8,16.1,11.4
San Antonio,83.76,74.78,60.97,59.19,84.88,51.41,11.0,13.1,16.8,20.8,25.0,28.1,29.3,29.7,26.6,21.8,16.2,11.6
Charlotte,84.46,77.18,72.08,67.05,84.39,30.21,5.1,7.2,11.3,15.8,20.3,24.7,26.4,25.8,22.2,16.3,11.1,6.3
Seattle,85.38,78.46,75.46,32.90,84.10,42.03,5.4,6.2,7.9,10.1,13.2,15.9,18.6,18.8,16.2,11.4,7.4,4.7
Minneapolis,83.47,69.91,62.35,77.94,83.79,40.36,-9.1,-6.2,0.4,8.6,15.1,20.4,23.2,21.7,16.7,9.4,0.9,-6.8


Climatewise Minneapolis is in the cold Cluster 5, which our stakeholders likely wouldn't like, while San Antonio and Austin (Cluster 1) with their South-Mediterranean climate may be great for holidays, but maybe not-so-much for productive work. This leaves us with Charlotte and Seattle: but Charlotte's air pollution value of 67.05 is not ideal. So, looks like our candidate in this first iteration of the project is... Seattle!

Before we move to selecting ideal neighborhoods in Seattle, let's investigate this "Quality of Life" index a bit. Can we find a relationship with our other data features? We'll run a quick and decidedly simple multiple regression to find out. Given the low amount of rows we have and the fact that our goal is only to see effect sizes, not making predictions, we'll refrain from splitting the data to training and test sets.

In [30]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
scale=StandardScaler()
lm = LinearRegression()

# Our predictors, let's normalize them before fitting the model, so we can use 
# the coefficient magnitudes as indicators of effect size
Predictors = ['Purchase Power', 'Health Care', 'Pollution', 'Crime Rating', 'Jan', 'Apr', 'Jul', 'Oct']
X = us_cities[Predictors]
X_scaled = scale.fit_transform(X)

# Our target
y = us_cities['Quality of Life']

#fit model
lm.fit(X_scaled, y)

#Print results
print('R-squared:', lm.score(X_scaled, y))
print('\nIntercept: ', lm.intercept_)
pd.DataFrame(lm.coef_, index=Predictors, columns=['Coefficients'])


R-squared: 0.6347392385623969

Intercept:  77.40608695652173


,Coefficients
Purchase Power,0.669500
Health Care,2.368927
Pollution,-0.539854
Crime Rating,-6.009810
Jan,-6.872101
Apr,10.282084
Jul,-2.811059
Oct,-0.611531


R-squared is only 0.63, even in this in-sample evaluation of our model, so looks like there are other factors in play as well. The data we gathered so far is not enough to fully explain the Quality of Life scores. A later iteration may involve sourcing other data vectors as well. Still, looking at the coefficients produced by the model fitting, interestingly the January and April mean temperatures have the biggest effect among our predictors, and - not so surprisingly - the Crime Rate is important as well.

# Finding the right neighborhood
We'll be using the Foursquare API to get information about the neighborhoods of our candidate city.

In [31]:
CLIENT_ID = '4GBLFXNTJYAQ1M1MVPD3UA1E0TWT05EW4ASRANGSGBND2CZ1' # your Foursquare ID
CLIENT_SECRET = 'WHJ1FT2UEFNEG5PY1CTZEATX0HSZXGWLYEOV3ZAI12QP4OEW' # your Foursquare Secret
ACCESS_TOKEN = 'LYJ3PGZEZDWMVTQHZ3EVPJEK3VIG5M3I3NDS51OSFTZE02UJ' # Auth Access Token
VERSION = '20180604'
LIMIT = 100

Let's see Seattle on the map.

In [59]:
seattle_idx = np.where(city_avg_temp_clustering.index == 'Seattle')[0][0]
seattle_coords = coordinates[seattle_idx]

seattle_map = folium.Map(seattle_coords, zoom_start=11)
seattle_map

Next we'll grab GeoJSON data for Seattle neighborhoods from a GitHub repo.

In [145]:
#!curl -O https://raw.githubusercontent.com/seattleio/seattle-boundaries-data/master/data/neighborhoods.geojson
!curl -O https://raw.githubusercontent.com/seattleflu/website/master/current/src/components/FluMap/geometry/seattle-neighborhoods.geojson
seattle_neighborhoods = r'seattle-neighborhoods.geojson'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 19337  100 19337    0     0  19337      0  0:00:01 --:--:--  0:00:01 49582


Let's get the names of all Seattle neighborhoods:

In [146]:
with open(seattle_neighborhoods) as f:
    seattle_neighborhoods_json = json.load(f)
seattle_neighborhoods_df = pd.io.json.json_normalize(seattle_neighborhoods_json["features"])
seattle_neighborhoods_df.head()

,type,properties.STATEFP,properties.COUNTYF,properties.TRACTCE,properties.AFFGEOI,properties.GEOID,properties.NAME,properties.LSAD,properties.ALAND,properties.AWATER,properties.rowID,properties.CRA_NAM,properties.NEIGHBO,properties.PUMA5CE,geometry.type,geometry.coordinates
0,Feature,1400000US53033003000,033,003000,1400000US53033003000,53033003000,30,CT,1493249,0,1,Whittier Heights,Ballard,11601,MultiPolygon,"[[[[-122.366169, 47.672347], [-122.366179, 47...."
1,Feature,1400000US53033005801,033,005801,1400000US53033005801,53033005801,58.01,CT,1814668,264894,2,Interbay,Magnolia/Queen Anne,11603,MultiPolygon,"[[[[-122.351489, 47.624568], [-122.347648, 47...."
2,Feature,1400000US53033007600,033,007600,1400000US53033007600,53033007600,76,CT,571879,0,3,Miller Park,East,11604,MultiPolygon,"[[[[-122.285036, 47.64752], [-122.256908, 47.6..."
3,Feature,1400000US53033000200,033,000200,1400000US53033000200,53033000200,2,CT,3286278,0,4,Olympic Hills/Victory Heights,North,11602,MultiPolygon,"[[[[-122.290876, 47.704783], [-122.290704, 47...."
4,Feature,1400000US53033009300,033,009300,1400000US53033009300,53033009300,93,CT,9429073,719985,5,Duwamish/SODO,Greater Duwamish,11605,MultiPolygon,"[[[[-122.278305, 47.531541], [-122.278083, 47...."


In [147]:
seattle_nnames = seattle_neighborhoods_df['properties.NEIGHBO'].unique()
seattle_nnames

array(['Ballard', 'Magnolia/Queen Anne', 'East', 'North',
       'Greater Duwamish', 'Southeast', 'Central', 'Northwest',
       'Lake Union', 'Downtown', 'Southwest', 'Delridge Neighborhoods',
       'Northeast'], dtype=object)

Let's also get the coordinates of these neighborhoods. We'll use our `do_geocode` function again, but some neighborhoods cannot be geolocated by the service we use. In these cases, fall back to the first point we find in the GeoJSON file for that neighborhood.

In [149]:
def get_neighborhood_first_coords(geojson_df, neighborhood):
    row = geojson_df[geojson_df['properties.NEIGHBO']==neighborhood].iloc[0]
    if row['geometry.type']=='Polygon':
        lon, lat = row['geometry.coordinates'][0][0]
    elif row['geometry.type']=='MultiPolygon':
        lon, lat = row['geometry.coordinates'][0][0][0]
    return [lat, lon]

seattle_n_coords = []
for neighborhood in seattle_nnames: 
    location = do_geocode(geolocator, f'{neighborhood}, Seattle, USA') 
    try:
        seattle_n_coords.append([location.latitude, location.longitude])
    except:
        # Neighborhoods cannot be geolocated: fall back to the first point in GeoJSON        
        seattle_n_coords.append(get_neighborhood_first_coords(seattle_neighborhoods_df, neighborhood))        

We'll use a slightly modified version of the function used in the course labs to get the venues.

In [153]:
def get_venues_near_neighborhoods(neighborhoods, coords_list, category=None, radius=1000):
    
    venues_list=[]
    section_query = '' if not category else f'&section={category}'
    for name, coords in zip(neighborhoods, coords_list):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            coords[0], 
            coords[1],
            section_query,
            radius, 
            LIMIT)
            
        # make the GET request
        response = requests.get(url).json()['response']
        try:
            results = response['groups'][0]['items']
        except:
            print(f'{name}: no results. Response: \n{response}')
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            coords[0],
            coords[1], 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [154]:
seattle_food = get_venues_near_neighborhoods(seattle_nnames, seattle_n_coords, category='food')
seattle_food
    

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ballard,47.676507,-122.386223,Copine,47.675741,-122.387404,French Restaurant
1,Ballard,47.676507,-122.386223,Cafe Besalu,47.671971,-122.387755,Bakery
2,Ballard,47.676507,-122.386223,Tall Grass Bakery,47.671982,-122.387690,Bakery
3,Ballard,47.676507,-122.386223,Kimchi House,47.671372,-122.387763,Korean Restaurant
4,Ballard,47.676507,-122.386223,Mr. Gyros,47.669304,-122.382018,Mediterranean Restaurant
...,...,...,...,...,...,...,...
752,Northeast,47.603832,-122.330062,A+ Hong Kong Kitchen,47.598529,-122.326745,Cantonese Restaurant
753,Northeast,47.603832,-122.330062,TASTE,47.607198,-122.338251,New American Restaurant
754,Northeast,47.603832,-122.330062,Umma's Lunch Box,47.609110,-122.334161,Korean Restaurant
755,Northeast,47.603832,-122.330062,Harbor City Restaurant,47.598377,-122.323382,Chinese Restaurant


757 places to eat at in Seattle! Let's count them by neighborhood and show their frequency on a choropleth map.

In [155]:
seattle_food_counts = seattle_food.groupby('Neighborhood')['Venue'].count().reset_index()
seattle_food_counts

,Neighborhood,Venue
0,Ballard,83
1,Central,60
2,Delridge Neighborhoods,5
3,Downtown,100
4,East,35
5,Greater Duwamish,34
6,Lake Union,32
7,Magnolia/Queen Anne,20
8,North,44
9,Northeast,100


In [190]:
seattle_map = folium.Map(seattle_coords, zoom_start=11)
folium.Choropleth(
    geo_data=seattle_neighborhoods,
    data=seattle_food_counts,
    columns=['Neighborhood', 'Venue'],
    key_on='feature.properties.NEIGHBO',    
    fill_color='YlGn', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    name='Food venues',
    legend_name='Food venues'
).add_to(seattle_map)

# display map
seattle_map


Now that we visualized food venue frequency, let's cover other categories as well. We'll write a function to do what we did with the food category and will call it for the other categories.

In [ ]:
categories = ['drinks', 'coffee', 'shops', 'arts', 'outdoors', 'sights', 'trending', 'topPicks']
venue_counts = {}    
    
def get_venue_counts_by_category(neighborhoods, nhood_coords, category):
    venues = get_venues_near_neighborhoods(neighborhoods, nhood_coords, category=category)
    venue_counts=venues.groupby('Neighborhood')['Venue'].count().reset_index()
    return venue_counts

for category in categories:
    venue_counts[category] = get_venue_counts_by_category(seattle_nnames, seattle_n_coords, category)

In [183]:
def choropleth_by_venue_category(folium_map, geo_json, neighborhood_key, venue_counts, category):
    return folium.Choropleth(
        geo_data=geo_json,
        data=venue_counts,
        columns=['Neighborhood', 'Venue'],
        key_on=neighborhood_key,    
        fill_color='YlGn', 
        fill_opacity=0.7, 
        line_opacity=0.2,
        name=f'{category} venues',
        legend_name=f'{category} venues'
    )
    
maps = {}
for category in categories:
    choro = choropleth_by_venue_category(seattle_map, 
                                         seattle_neighborhoods, 
                                         'feature.properties.NEIGHBO',
                                         venue_counts[category],
                                         category)
    maps[category] = folium.Map(seattle_coords, zoom_start=11.2)
    choro.add_to(maps[category])

We'll end this notebook and research iteration with the frequency maps of coffee shops, sights and top picks of Foursquare users. Hopefully this will help our imaginary software company make their decision.

In [185]:
maps['coffee']

In [186]:
maps['sights']

In [187]:
maps['topPicks']

Based on these, many neighborhoods look great: e.g. Ballard on the Northwestern part of the city would be an excellent choice for both office and for the developers to buy a house in. If they're still unsure, we can perform an additional iteration, adding new data, taking into consideration things missed here, such as cost of living, property prices, other vectors of climate such as yearly sunshine hours, precipitation and so on.